# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- EC - exercise count
- EI - exercise intensity - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- RN - running
- GM - gaming


(Separate message about EI)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [1]:
import numpy as np
import pandas as pd

from app.preparation.prepare_data import prepare_data
from app.constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER, DESIRED_MEAN_VALUE

rep_data, wide_use_data, feedback_data = prepare_data(PATH, DESIRED_MEAN_VALUE)
month, mean, std, df = wide_use_data
df

c:\Users\San\Documents\CS projects\time_counter\app\preparation\utils\initial_preparation.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  month = df.iloc[0][0]


,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,RN,GM
0,1,relax,0,20,55,75,0,0,0,0,0


In [2]:
rep_data

([0, 0, 1, 1, 75, 1, '1'],
 [['OD', 'RN', 'GM'],
  0,
  [0.0, 0.0, 0.0],
  [0, 0, 0],
  {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}])